In [18]:
# pip install langchain_community

In [19]:
# pip install langchain_openai

In [20]:
# pip install pysqlite3

In [21]:
# pip install Chroma

In [23]:
# pip install pymupdf

In [24]:
# pip install langchain-google-genai

In [25]:
import os
import sys

# Import classes from modules
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI


__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [26]:
# Generate API KEY from Gemini and OpenAI website and define as a variable.
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEYS"
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEYS"

In [27]:
# Define the path to your local PDF file
local_path = "ข้อมูลยา 50 ชนิด.pdf"

# Load and split the PDF file
if local_path:
    loader = PyMuPDFLoader(file_path=local_path)
    data = loader.load_and_split()
else:
    print("Upload a PDF file")
    sys.exit()

In [28]:
# data

In [29]:
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [30]:
# texts

In [31]:
# This part is used for embedding the docs and storing them into Vector DB and initializing the retriever.
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

In [32]:
custom_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are an AI thai language model assistant.
    You are an expert at answering questions about medicine.
    Answer the question based ONLY on the following context.


    {context}


    Original question: {question}""",
)

In [33]:
# Create the language model
llm1 = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

chain = ConversationalRetrievalChain.from_llm(
    llm=llm1,
    retriever=docsearch.as_retriever(),
    combine_docs_chain_kwargs={"prompt": custom_template}
)

In [34]:
chat_history = []
query = None  # Initialize query to avoid potential reference error

while True:
    if not query:
        query = input("User: ")
    if query in ['quit', 'q', 'exit']:
        break
    result = chain.invoke({"question": query, "chat_history": chat_history})
    print("Chatbot:", result['answer'])

    chat_history.append((query, result['answer']))
    query = None

User: พาราเซตามอลเก็บรักษาอย่างไร
Chatbot: เก็บในที่แห้งและเย็น, ห่างจากแสงแดด 

User: q
